# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
# Monte the files from MyDrive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# install packages for hugging face
!pip install sentence-transformers faiss-cpu tqdm langdetect nltk



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00
 

In [ ]:
# Import all the packages
import numpy as np
import json
from collections import Counter
from sentence_transformers import SentenceTransformer
import pickle
import faiss
from langdetect import detect
from tqdm import tqdm
import nltk
import re
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Analyse train-claims and evidences

train_claims_path = '/content/drive/MyDrive/Colab Notebooks/data/train-claims.json'
dev_claims_path = '/content/drive/MyDrive/Colab Notebooks/data/dev-claims.json'
evidence_path = '/content/drive/MyDrive/Colab Notebooks/data/evidence.json'

def analyze_data(train_claims_path, dev_claims_path, evidence_path):
  # Load the datasets
  with open(train_claims_path, 'r') as f:
    train_claims = json.load(f)
  with open(dev_claims_path, 'r') as f:
    dev_claims = json.load(f)
  with open(evidence_path, 'r') as f:
    evidences = json.load(f)

  #  Analyze train-claims.json
  print("=== Train Claims Analysis ===")
  print(f"Total train claims: {len(train_claims)}")
  # Claim length (in characters and tokens) & evidence count
  claim_lengths_char = []
  claim_lengths_token = []
  evidence_counts = []
  claim_labels = Counter()

  for claim in train_claims.values():
    claim_text = claim["claim_text"]
    claim_lengths_char.append(len(claim_text))
    claim_lengths_token.append(len(claim_text.split()))
    evidence_counts.append(len(claim.get("evidences", [])))
    claim_labels[claim["claim_label"]] += 1

  print(f"Max claim length (characters): {max(claim_lengths_char)}")
  print(f"Min claim length (characters): {min(claim_lengths_char)}")
  print(f"Mean claim length (characters): {np.mean(claim_lengths_char):.1f}")
  print(f"Max claim length (tokens): {max(claim_lengths_token)}")
  print(f"Min claim length (tokens): {min(claim_lengths_token)}")
  print(f"Mean claim length (tokens): {np.mean(claim_lengths_token):.1f}")

  print(f"Max evidence count: {max(evidence_counts)}")
  print(f"Min evidence count: {min(evidence_counts)}")
  print(f"Mean evidence count: {np.mean(evidence_counts):.2f}")

  print("Claim label distribution:")
  for label, count in claim_labels.items():
    print(f"  {label}: {count} ({count / len(train_claims):.2%})")

  # Analyze dev-claims.json
  print("=== Dev Claims Analysis ===")
  print(f"Total dev claims: {len(dev_claims)}")
  # Claim length (in characters and tokens) & evidence count
  claim_lengths_char = []
  claim_lengths_token = []
  evidence_counts = []
  claim_labels = Counter()

  for claim in dev_claims.values():
    claim_text = claim["claim_text"]
    claim_lengths_char.append(len(claim_text))
    claim_lengths_token.append(len(claim_text.split()))
    evidence_counts.append(len(claim.get("evidences", [])))
    claim_labels[claim["claim_label"]] += 1

  print(f"Max claim length (characters): {max(claim_lengths_char)}")
  print(f"Min claim length (characters): {min(claim_lengths_char)}")
  print(f"Mean claim length (characters): {np.mean(claim_lengths_char):.1f}")
  print(f"Max claim length (tokens): {max(claim_lengths_token)}")
  print(f"Min claim length (tokens): {min(claim_lengths_token)}")
  print(f"Mean claim length (tokens): {np.mean(claim_lengths_token):.1f}")

  print(f"Max evidence count: {max(evidence_counts)}")
  print(f"Min evidence count: {min(evidence_counts)}")
  print(f"Mean evidence count: {np.mean(evidence_counts):.2f}")

  print("Claim label distribution:")
  for label, count in claim_labels.items():
    print(f"  {label}: {count} ({count / len(dev_claims):.2%})")
  # Analyze evidences.json
  print("\n=== Evidence Analysis ===")
  print(f"Total evidences: {len(evidences)}")
  evidence_lengths_char = [len(text) for text in evidences.values()]
  evidence_lengths_token = [len(text.split()) for text in evidences.values()]

  print(f"Max evidence length (characters): {max(evidence_lengths_char)}")
  print(f"Min evidence length (characters): {min(evidence_lengths_char)}")
  print(f"Mean evidence length (characters): {np.mean(evidence_lengths_char):.1f}")
  print(f"Max evidence length (tokens): {max(evidence_lengths_token)}")
  print(f"Min evidence length (tokens): {min(evidence_lengths_token)}")
  print(f"Mean evidence length (tokens): {np.mean(evidence_lengths_token):.1f}")

# Example usage:
analyze_data(train_claims_path, dev_claims_path, evidence_path)


=== Train Claims Analysis ===
Total train claims: 1228
Max claim length (characters): 332
Min claim length (characters): 26
Mean claim length (characters): 123.0
Max claim length (tokens): 67
Min claim length (tokens): 4
Mean claim length (tokens): 20.1
Max evidence count: 5
Min evidence count: 1
Mean evidence count: 3.36
Claim label distribution:
  DISPUTED: 124 (10.10%)
  REFUTES: 199 (16.21%)
  SUPPORTS: 519 (42.26%)
  NOT_ENOUGH_INFO: 386 (31.43%)
=== Dev Claims Analysis ===
Total dev claims: 154
Max claim length (characters): 406
Min claim length (characters): 31
Mean claim length (characters): 128.0
Max claim length (tokens): 65
Min claim length (tokens): 4
Mean claim length (tokens): 21.1
Max evidence count: 5
Min evidence count: 1
Mean evidence count: 3.19
Claim label distribution:
  SUPPORTS: 68 (44.16%)
  NOT_ENOUGH_INFO: 41 (26.62%)
  REFUTES: 27 (17.53%)
  DISPUTED: 18 (11.69%)

=== Evidence Analysis ===
Total evidences: 1208827
Max evidence length (characters): 3148
Min ev

In [ ]:
train_claims_path = '/content/drive/MyDrive/Colab Notebooks/data/train-claims.json'
dev_claims_path = '/content/drive/MyDrive/Colab Notebooks/data/dev-claims.json'
evidence_path = '/content/drive/MyDrive/Colab Notebooks/data/evidence.json'

In [ ]:
def is_english(text: str, threshold: float = 0.8) -> bool:
  # 清理文本，仅保留字母和空格
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  if len(text) == 0:  # 如果文本清理后为空，返回False
    return False
  # 计算英文字符占比
  english_char_count = sum(1 for char in text if char.isalpha())
  return (english_char_count / len(text)) >= threshold

def clean_and_split(eid, text):
  result_ids = []
  result_texts = []
  if not is_english(text):
    return result_ids, result_texts
  sentences = sent_tokenize(text)
  for i, sent in enumerate(sentences):
    sent = sent.lower()
    sent = re.sub(r'[^a-z0-9\s]', '', sent)  # 去除标点
    sent = re.sub(r'\s+', ' ', sent).strip()
    if len(sent.split()) >= 5:  # 可选：过滤太短的文本
      result_ids.append(f"{eid}_s{i}")  # 用原始的 eID 加上句子索引
      result_texts.append(sent)
  return result_ids, result_texts

In [ ]:
encode = False # Variable to control whether encode the evidence
# Load the sentence-BERT (all-MiniLM-L6-v2)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
if (encode):
  # 加载 evidence 数据集
  with open(evidence_path, 'r') as f:
    evidence_dict = json.load(f)

  # 清洗和分句后的结果
  cleaned_evidence_ids = []
  cleaned_evidence_texts = []
  original_evidence_ids = []  # 记录每个分句所属的原始 evidence_id

  # 遍历 evidence 数据并进行清洗和分句
  for eid, text in tqdm(evidence_dict.items(),desc="Cleaning evidences"):
    cleaned_ids, cleaned_texts = clean_and_split(eid, text)
    cleaned_evidence_ids.extend(cleaned_ids)
    cleaned_evidence_texts.extend(cleaned_texts)
    original_evidence_ids.extend([eid] * len(cleaned_ids))  # 每个分句都记录原始的 eID

  # 使用 Sentence-BERT 对清洗后的分句进行编码
  evidence_embeddings = model.encode(
    cleaned_evidence_texts,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
  )

  # 保存编码后的嵌入（embeddings）
  np.save("/content/drive/MyDrive/Colab Notebooks/Word_embeddings/evidence_embeddings.npy", evidence_embeddings)

  # 保存分句后的 evidence_ids 和 texts 以及对应的原始 evidence_id 映射
  with open("/content/drive/MyDrive/Colab Notebooks/Word_embeddings/evidence_meta.pkl", "wb") as f:
      pickle.dump((cleaned_evidence_ids, cleaned_evidence_texts, original_evidence_ids), f)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Load numpy embeddings
evidence_embeddings = np.load("/content/drive/MyDrive/Colab Notebooks/Word_embeddings/evidence_embeddings.npy")

# Load evidence_ids, evidence_texts, and original_evidence_ids
with open("/content/drive/MyDrive/Colab Notebooks/Word_embeddings/evidence_meta.pkl", "rb") as f:
  evidence_ids, evidence_texts, original_evidence_ids = pickle.load(f)


dimension = evidence_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine similarity if normalized
index.add(evidence_embeddings)


def clean_claim(claim: str) -> str:
  # 小写化
  claim = claim.lower()
  # 去除标点符号
  claim = re.sub(r'[^a-z0-9\s]', '', claim)
  # 去除多余空格
  claim = re.sub(r'\s+', ' ', claim).strip()
  # 如果需要，你也可以添加去除停用词的步骤
  # claim = " ".join([word for word in claim.split() if word not in stopwords])
  return claim

# Create mapping
evidence_dict = dict(zip(evidence_ids, evidence_texts))

def retrieve_evidence(claim_text, top_k=5):
  # 清洗 claim
  cleaned_claim = clean_claim(claim_text)
  # 编码 claim
  claim_embedding = model.encode([cleaned_claim], convert_to_numpy=True, normalize_embeddings=True)
  # 执行检索
  scores, indices = index.search(claim_embedding, top_k*10)

  # 准备去重筛选
  seen_original_ids = set()
  results = []

  for i in indices[0]:
    eid = evidence_ids[i]
    text = evidence_dict[eid]
    original_id = original_evidence_ids[i]

    if original_id not in seen_original_ids:
      results.append((original_id, eid, text))
      seen_original_ids.add(original_id)

    if len(results) >= top_k:
      break

  return results

In [ ]:
claim = "[South Australia] has the most expensive electricity in the world."
with open(evidence_path, 'r') as f:
  evidences = json.load(f)
retrieved = retrieve_evidence(claim, top_k=7)

for org_eid,_,_ in retrieved:
  print(f"{org_eid}: {evidences[org_eid]}\n")

evidence-572512: "South Australia has the highest power prices in the world".

evidence-67732: [citation needed] South Australia has the highest retail price for electricity in the country.

evidence-452156: Energy in Australia is the production in Australia of energy and electricity, for consumption or export.

evidence-1061888: Australia has one of the fastest deployment rates of renewable energy worldwide.

evidence-780332: Industrialised countries such as Canada, the US, and Australia are among the highest per capita consumers of electricity in the world, which is possible thanks to a widespread electrical distribution network.

evidence-666596: "Renewable energy now cheaper than new fossil fuels in Australia".

evidence-952729: In September 2005 Flannery said, "There are hot rocks in South Australia that potentially have enough embedded energy in them to run Australia's economy for the best part of a century".



In [ ]:
# Evaluate the evidence retrieval method
with open(dev_claims_path, 'r') as f:
  dev_claims = json.load(f)


claim_ids = list(dev_claims.keys())
top_k = 7

recalls = []
for cid in tqdm(claim_ids,desc="Evaluating"):
  truth = set(dev_claims[cid]["evidences"])
  retrieved = [original_id for original_id,_,_ in retrieve_evidence(dev_claims[cid]["claim_text"], top_k)]

  # Precision & Recall
  hit = len(truth & set(retrieved))
  recalls.append(hit / len(truth))

print("=== Evidence Retrieval Evaluation ===")
print(f"Average Recall@{top_k}: {np.mean(recalls):.2%}")

Evaluating: 100%|██████████| 154/154 [00:32<00:00,  4.69it/s]

=== Evidence Retrieval Evaluation ===
Average Recall@10: 31.62%


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*